In [5]:
pip install joblib

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [1]:
pip install confluent_kafka

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [41]:
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
import joblib

In [5]:
# Charger le dataset
df = pd.read_csv("tweets_sentiment.csv")

# Afficher directement le DataFrame sous forme de tableau
df.head(20)  # Affiche les 20 premières lignes du DataFrame sous forme de tableau


tweet_id airline_sentiment  airline_sentiment_confidence  \
0   570306133677760513           neutral                        1.0000   
1   570301130888122368          positive                        0.3486   
2   570301083672813571           neutral                        0.6837   
3   570301031407624196          negative                        1.0000   
4   570300817074462722          negative                        1.0000   
5   570300767074181121          negative                        1.0000   
6   570300616901320704          positive                        0.6745   
7   570300248553349120           neutral                        0.6340   
8   570299953286942721          positive                        0.6559   
9   570295459631263746          positive                        1.0000   
10  570294189143031808           neutral                        0.6769   
11  570289724453216256          positive                        1.0000   
12  570289584061480960          positive                        1.0000   
13  570287408438120448          positive                        0.6451   
14  570285904809598977          positive                        1.0000   
15  570282469121007616          negative                        0.6842   
16  570277724385734656          positive                        1.0000   
17  570276917301137409          negative                        1.0000   
18  570270684619923457          positive                        1.0000   
19  570267956648792064          positive                        1.0000   

   negativereason  negativereason_confidence         airline  \
0             NaN                        NaN  Virgin America   
1             NaN                     0.0000  Virgin America   
2             NaN                        NaN  Virgin America   
3      Bad Flight                     0.7033  Virgin America   
4      Can't Tell                     1.0000  Virgin America   
5      Can't Tell                     0.6842  Virgin America   
6             NaN                     0.0000  Virgin America   
7             NaN                        NaN  Virgin America   
8             NaN                        NaN  Virgin America   
9             NaN                        NaN  Virgin America   
10            NaN                     0.0000  Virgin America   
11            NaN                        NaN  Virgin America   
12            NaN                        NaN  Virgin America   
13            NaN                        NaN  Virgin America   
14            NaN                        NaN  Virgin America   
15    Late Flight                     0.3684  Virgin America   
16            NaN                        NaN  Virgin America   
17     Bad Flight                     1.0000  Virgin America   
18            NaN                        NaN  Virgin America   
19            NaN                        NaN  Virgin America   

   airline_sentiment_gold             name negativereason_gold  retweet_count  \
0                     NaN          cairdin                 NaN              0   
1                     NaN         jnardino                 NaN              0   
2                     NaN       yvonnalynn                 NaN              0   
3                     NaN         jnardino                 NaN              0   
4                     NaN         jnardino                 NaN              0   
5                     NaN         jnardino                 NaN              0   
6                     NaN       cjmcginnis                 NaN              0   
7                     NaN            pilot                 NaN              0   
8                     NaN         dhepburn                 NaN              0   
9                     NaN       YupitsTate                 NaN              0   
10                    NaN  idk_but_youtube                 NaN              0   
11                    NaN     HyperCamiLax                 NaN              0   
12                    NaN     HyperCamiLax                 Na

In [10]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder

# Charger le dataset
df = pd.read_csv("tweets_sentiment.csv")

# Sélection des colonnes numériques et catégorielles
numerical_cols = df.select_dtypes(include=['int64', 'float64']).columns
categorical_cols = df.select_dtypes(include=['object', 'category']).columns

# Remplir les valeurs manquantes des colonnes numériques avec la moyenne
df[numerical_cols] = df[numerical_cols].fillna(df[numerical_cols].mean())

# Remplir les valeurs manquantes des colonnes catégorielles avec le mode (valeur la plus fréquente)
df[categorical_cols] = df[categorical_cols].apply(lambda col: col.fillna(col.mode()[0]))

# Encodage LabelEncoder sur les colonnes catégorielles
df[categorical_cols] = df[categorical_cols].apply(lambda col: LabelEncoder().fit_transform(col.astype(str)))

# Vérification finale : plus aucun NaN
print(df.isna().sum())
df.head(20)

tweet_id                        0
airline_sentiment               0
airline_sentiment_confidence    0
negativereason                  0
negativereason_confidence       0
airline                         0
airline_sentiment_gold          0
name                            0
negativereason_gold             0
retweet_count                   0
text                            0
tweet_coord                     0
tweet_created                   0
tweet_location                  0
user_timezone                   0
dtype: int64


tweet_id  airline_sentiment  airline_sentiment_confidence  \
0   570306133677760513                  1                        1.0000   
1   570301130888122368                  2                        0.3486   
2   570301083672813571                  1                        0.6837   
3   570301031407624196                  0                        1.0000   
4   570300817074462722                  0                        1.0000   
5   570300767074181121                  0                        1.0000   
6   570300616901320704                  2                        0.6745   
7   570300248553349120                  1                        0.6340   
8   570299953286942721                  2                        0.6559   
9   570295459631263746                  2                        1.0000   
10  570294189143031808                  1                        0.6769   
11  570289724453216256                  2                        1.0000   
12  570289584061480960                  2                        1.0000   
13  570287408438120448                  2                        0.6451   
14  570285904809598977                  2                        1.0000   
15  570282469121007616                  0                        0.6842   
16  570277724385734656                  2                        1.0000   
17  570276917301137409                  0                        1.0000   
18  570270684619923457                  2                        1.0000   
19  570267956648792064                  2                        1.0000   

    negativereason  negativereason_confidence  airline  \
0                3                   0.638298        5   
1                3                   0.000000        5   
2                3                   0.638298        5   
3                0                   0.703300        5   
4                1                   1.000000        5   
5                1                   0.684200        5   
6                3                   0.000000        5   
7                3                   0.638298        5   
8                3                   0.638298        5   
9                3                   0.638298        5   
10               3                   0.000000        5   
11               3                   0.638298        5   
12               3                   0.638298        5   
13               3                   0.638298        5   
14               3                   0.638298        5   
15               7                   0.368400        5   
16               3                   0.638298        5   
17               0                   1.000000        5   
18               3                   0.638298        5   
19               3                   0.638298        5   

    airline_sentiment_gold  name  negativereason_gold  retweet_count   text  \
0                        0  4049                    4              0  10006   
1                        0  5395                    4              0  10157   
2                        0  7678                    4              0   9905   
3                        0  5395                    4              0  10119   
4                        0  5395                    4              0  10034   
5                        0  5395                    4              0  10166   
6                        0  4214                    4              0  10238   
7                        0  6582                    4              0   9977   
8                        0  4476                    4              0  14068   
9                        0  3455                    4              0  10118   
10                       0  5113                    4              0  10065   
11                       0  1329                    4              0   9895   
12                       0  1329                    4              0   9996   
13                       0  6236                    4              0   9839   
14                       0  7034  

In [21]:
# Mapping inversé : 0 -> neutral, 1 -> negative, 2 -> positive
df['label'] = df['airline_sentiment'].map({0: 'neutral', 1: 'negative', 2: 'positive'})

# Vérifier les résultats
print(df[['airline_sentiment', 'label']].head(10))


   airline_sentiment     label
0                  1  negative
1                  2  positive
2                  1  negative
3                  0   neutral
4                  0   neutral
5                  0   neutral
6                  2  positive
7                  1  negative
8                  2  positive
9                  2  positive


In [36]:
# Convertir toutes les valeurs de X_train en chaînes de caractères
X_train = X_train.astype(str)
X_test = X_test.astype(str)
# 1. Transformation des données textuelles en vecteurs (TF-IDF)
vectorizer = TfidfVectorizer(max_features=5000)
X_train_vect = vectorizer.fit_transform(X_train)  # Transformation des données d'entraînement
X_test_vect = vectorizer.transform(X_test)  # Transformation des données de test
X_train,X_test,y_train,y_test= train_test_split(X,y, test_size=0.3, random_state=42)

In [37]:
# 2. Entraîner le modèle Random Forest
model = RandomForestClassifier()
model.fit(X_train_vect, y_train)

RandomForestClassifier()

In [38]:
# 3. Prédictions sur les données de test
y_pred = model.predict(X_test_vect)

# 4. Affichage du rapport de classification
print(classification_report(y_test, y_pred))

# 5. Créer un tableau avec les résultats (texte et sentiment prédit)
df_results = pd.DataFrame({
    "text": X_test,  # Tu peux remplacer X_test par df['text'] si tu veux inclure le texte brut
    "predicted_sentiment": y_pred
})

# Afficher les premières lignes des résultats
print(df_results.head())


              precision    recall  f1-score   support

    negative       0.67      0.00      0.01       884
     neutral       0.64      1.00      0.78      2814
    positive       0.95      0.03      0.06       694

    accuracy                           0.65      4392
   macro avg       0.76      0.34      0.28      4392
weighted avg       0.70      0.65      0.51      4392

       text predicted_sentiment
4794   6846             neutral
10480  8636             neutral
8067   4350             neutral
8880   3622             neutral
8292   3010             neutral


In [39]:
from sklearn.svm import SVC
from sklearn.metrics import classification_report
# 6. Entraîner le modèle SVC
model = SVC()
model.fit(X_train_vect, y_train)

# 7. Prédictions
y_pred = model.predict(X_test_vect)

# 8. Rapport
print(classification_report(y_test, y_pred))

# 9. Affichage des résultats
df_results = pd.DataFrame({
    "text": X_test.values,
    "predicted_sentiment": y_pred
})

              precision    recall  f1-score   support

    negative       0.67      0.00      0.01       884
     neutral       0.64      1.00      0.78      2814
    positive       0.95      0.03      0.06       694

    accuracy                           0.65      4392
   macro avg       0.76      0.34      0.28      4392
weighted avg       0.70      0.65      0.51      4392



In [42]:
# Enregistrer le modèle entraîné
joblib.dump(model, 'random_forest_model.pkl')

['random_forest_model.pkl']